In [3]:
import os
import warnings
import logging

# Hide all info/warning logs from TensorFlow, PyTorch, and Transformers
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Suppress Python warnings
warnings.filterwarnings("ignore")

# Suppress Hugging Face and Pydantic warnings
logging.getLogger("transformers").setLevel(logging.ERROR)
logging.getLogger("pydantic").setLevel(logging.ERROR)
logging.getLogger("absl").setLevel(logging.ERROR)
logging.getLogger("tensorflow").setLevel(logging.ERROR)

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Path to model weights
model_path = "/kaggle/input/bert-summarizer-gpu-final/bert_summarizer_gpu_final"

# Load tokenizer from original pretrained model
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Load fine-tuned model from your saved directory
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print(f"✅ Model and tokenizer loaded successfully on {device}")


✅ Model and tokenizer loaded successfully on cpu


In [7]:
from transformers import BertTokenizer, EncoderDecoderModel
import torch
import warnings
import logging

# Suppress all unnecessary warnings
warnings.filterwarnings("ignore")
logging.getLogger("transformers").setLevel(logging.ERROR)
from transformers import logging as hf_logging
hf_logging.set_verbosity_error()

# ===== 1. DEVICE SETUP =====
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ===== 2. LOAD TOKENIZER AND MODEL =====
model_path = "/kaggle/input/bert-summarizer-gpu-final/bert_summarizer_gpu_final"

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = EncoderDecoderModel.from_pretrained(model_path)

# ===== 3. FIX CONFIG (if missing after loading) =====
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.eos_token_id = tokenizer.sep_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.config.max_length = 128
model.config.min_length = 30
model.config.num_beams = 4
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.early_stopping = True

model.to(device)
print(f"✅ Model and tokenizer loaded successfully on {device}")

# ===== 4. SUMMARIZATION LOOP =====
print("\n🧠 BERT Summarizer is ready! Type a paragraph to summarize (or type 'exit' to quit)\n")

while True:
    text = input("Enter text: ").strip()
    if text.lower() == "exit":
        print("👋 Exiting summarizer.")
        break
    if not text:
        print("⚠️ Please enter some text.")
        continue

    # Tokenize input
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True).to(device)

    # Generate summary
    with torch.no_grad():
        summary_ids = model.generate(
            inputs["input_ids"],
            num_beams=4,
            max_length=128,
            min_length=30,
            no_repeat_ngram_size=3,
            early_stopping=True
        )

    # Decode and print summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    print("\n📝 Summary:\n", summary, "\n")

✅ Model and tokenizer loaded successfully on cpu

🧠 BERT Summarizer is ready! Type a paragraph to summarize (or type 'exit' to quit)



Enter text:  Artificial Intelligence is transforming industries around the world.  From healthcare and education to finance and transportation, AI is being used to improve efficiency, reduce costs, and enhance decision-making.  However, experts warn that without proper regulation and ethical standards, AI could also lead to job displacement and privacy concerns.  Governments are now working to establish guidelines for responsible AI development.



📝 Summary:
 new : u. s. lawmakers say it's a " big problem " in the future " we need to improve our lives, " he says, he says he says it is " a big problem, " and he says 



Enter text:  The global climate crisis is one of the most pressing issues of our time. Human activities, particularly the burning of fossil fuels, deforestation, and industrial processes,  have significantly increased the concentration of greenhouse gases in the atmosphere, leading to global warming. This warming has caused severe disruptions in weather patterns,  rising sea levels, and more frequent and intense natural disasters. Governments and organizations around the world are working towards reducing emissions, increasing renewable energy use,  and implementing policies to mitigate the impacts of climate change and adapt to its effects.



📝 Summary:
 new : u. s. climate change is a major factor in warming and climate change, scientists say climate change can be a major threat to climate change in the united states, the u. n. says it's the most important issue in the world 



Enter text:  The Internet of Things (IoT) is a network of interconnected devices that communicate and exchange data over the internet.  These devices, which include everything from smart thermostats and wearable fitness trackers to industrial machinery and vehicles, are embedded with sensors, software, and other technologies to collect and share data.  IoT allows for greater automation, efficiency, and real-time decision-making across various industries, such as healthcare, manufacturing, and agriculture.  With the growing number of connected devices, IoT has the potential to transform everyday life, making it more connected and intelligent. However, there are challenges such as security, privacy, and data management that need to be addressed as the IoT ecosystem continues to expand.



📝 Summary:
 google's new technology will be based on the internet it's the first time it will be connected to the internet the technology is the first to be available in the u. s. and europe 



Enter text:  exit


👋 Exiting summarizer.
